# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.




In [2]:
import json

def get_config(config_file):
    with open(config_file) as f:
        config = json.load(f)
        
    return config

In [3]:
def organ_failure_history_points(config_file, failure):
    config = get_config(config_file)
    return config.get("Organ Failure History").get(failure)

In [4]:
def range_points(config_file, key, val):
    config = get_config(config_file)
    rules = config.get(key)
    
    for rule in rules:
        minimum = rule.get("min")
        maximum = rule.get("max")
        score = rule.get("points")
        if val >= minimum and val < maximum:
            return score
    
    return None
            

In [5]:
def age_points(config_file, age):
    return range_points(config_file, "Age", age)

In [6]:
def temp_points(config_file, temp):
    return range_points(config_file, "Temperature", temp)

In [7]:
def ph_points(config_file, ph):
    return range_points(config_file, "pH", ph)

In [8]:
def heartrate_points(config_file, hr):
    return range_points(config_file, "Heart Rate", hr)

In [9]:
def respiratory_points(config_file, rr):
    return range_points(config_file, "Respiratory Rate", rr)

In [10]:
def sodium_points(config_file, sodium):
    return range_points(config_file, "Sodium", sodium)

In [11]:
def potassium_points(config_file, potassium):
    return range_points(config_file, "Potassium", potassium)

In [12]:
def creatinine_points(config_file, renal_failure, creatinine):
    with open(config_file) as f:
        config = json.load(f)
    
    rules = config.get("Creatinine").get(renal_failure)

    for rule in rules:
        minimum = rule.get("min")
        maximum = rule.get("max")
        score = rule.get("points")
        if creatinine >= minimum and creatinine < maximum:
            return score
    
    return None

In [13]:
def hematocrit_points(config_file, hematocrit):
    return range_points(config_file, "Hematocrit", hematocrit)

In [14]:
def white_blood_points(config_file, white_blood):
    return range_points(config_file, "White Blood Count", white_blood)

In [15]:
def fio2_points(config_file, fio2, pao2, aa_gradient):
    with open(config_file) as f:
        config = json.load(f)
        
    fio2_rules = config.get("FiO2")
    for fio2_rule in fio2_rules:
        minimum = fio2_rule.get("min")
        maximum = fio2_rule.get("max")
        if fio2 >= minimum and fio2 < maximum:
            rules = fio2_rule.get("rules")
            basis = fio2_rule.get("basis")
#             print("{} {} {}".format(minimum,maximum,fio2))
    
    if basis == "PaO2":
        val = pao2
    elif basis == "A-a Gradient":
        val = aa_gradient
    else:
        val = None
#     print(rules)
#     print(basis)
    
    for rule in rules:
        minimum = rule.get("min")
        maximum = rule.get("max")
        points = rule.get("points")
        if val >= minimum and val < maximum:
            return points
        
    return None
    
            

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [16]:
def apache_ii(url):
    """(str) -> list
    The function will fetch the CSV file from the specified internet location, calculate Apache II scores for
    each row in the input, and return a list of all the Apache II scores.
    """
    
    results = []
    
    ### Your Code Here

    return results
    

### Testing your Function

The URL for the test data is:

You can verify your results by comparing them against this data:



In [74]:
import pandas as pd

In [75]:
df = pd.read_csv('TestPatients.csv')

In [114]:
c = 'apache.json'
p = 'https://hds5210-2020.s3.amazonaws.com/TestPatients.csv'
s = 'https://hds5210-2020.s3.amazonaws.com/Scores.csv'

df = pd.read_csv(p)

for idx, row in df.iterrows():
    total = 0
    total +=organ_failure_history_points(c, row['Organ Failure History'])
    total +=age_points(c, row['Age'])
    total +=temp_points(c, row['Temperature'])
    total +=ph_points(c, row['pH'])
    total +=heartrate_points(c, row['Heart Rate'])
    total +=respiratory_points(c,row['Respiratory Rate'])
    total +=sodium_points(c, row['Sodium'])
    total +=potassium_points(c, row['Potassium'])
    total +=creatinine_points(c, row['Acute Renal Failure'], row['Creatinine'])
    total +=hematocrit_points(c, row['Hematocrit'])
    total +=white_blood_points(c, row['White Blood Count'])
    total +=fio2_points(c, row['FiO2'],row['PaO2'],row['A-a Gradient'])
#     total +=(15-row['Glasgow Coma Scale'])
    total +=row['Glasgow Coma Scale']
    df.loc[idx,'TOTAL'] = total

scores = pd.read_csv(s)
df['SCORE'] = scores

In [115]:
df[df['TOTAL'] != df['SCORE']]

,Patient,Organ Failure History,Age,Temperature,pH,Heart Rate,Respiratory Rate,Sodium,Potassium,Creatinine,Acute Renal Failure,Hematocrit,White Blood Count,Glasgow Coma Scale,FiO2,PaO2,A-a Gradient,TOTAL,SCORE


In [112]:
df[df['Patient'] == 'E6127']

,Patient,Organ Failure History,Age,Temperature,pH,Heart Rate,Respiratory Rate,Sodium,Potassium,Creatinine,Acute Renal Failure,Hematocrit,White Blood Count,Glasgow Coma Scale,FiO2,PaO2,A-a Gradient,TOTAL,SCORE


In [101]:
fio2_points(c, 14, 70, 460)

1

In [102]:
df

,Patient,Organ Failure History,Age,Temperature,pH,Heart Rate,Respiratory Rate,Sodium,Potassium,Creatinine,Acute Renal Failure,Hematocrit,White Blood Count,Glasgow Coma Scale,FiO2,PaO2,A-a Gradient,TOTAL,SCORE
0,E3408,Emergency,6,27,7.3,174,10,170,7.8,1.6,Chronic Renal Failure,3,28,13,61,59,458,46.0,46
1,E1100,Elective,3,4,7.2,74,53,106,3.2,1.2,Acute Renal Failure,60,22,11,66,59,482,38.0,38
2,E8001,Nonoperative,62,28,7.1,56,7,177,0.4,1.6,Acute Renal Failure,38,50,7,57,74,527,46.0,46
3,E4369,Nonoperative,10,32,7.7,10,26,144,6.2,1.8,Chronic Renal Failure,56,50,9,41,61,217,37.0,37
4,E5772,Emergency,14,9,7.5,63,2,51,6.8,1.2,Acute Renal Failure,12,19,2,36,56,438,33.0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,E1297,Emergency,55,9,8.0,191,20,195,0.4,1.4,Acute Renal Failure,22,28,5,38,53,528,41.0,50
330,E7778,Elective,54,22,7.9,113,31,10,4.6,3.4,Chronic Renal Failure,2,32,1,53,58,393,32.0,41
331,E3688,None,24,5,7.0,17,45,67,0.4,1.8,Chronic Renal Failure,13,24,4,31,61,503,36.0,35
332,E9557,Emergency,33,3,8.0,149,3,59,0.8,3.6,Acute Renal Failure,2,6,14,63,52,488,57.0,38


In [103]:
df['TOTAL'].to_csv('Scores.csv', index=False)

In [108]:
c = 'apache.json'
p = 'https://hds5210-2020.s3.amazonaws.com/TestPatients.csv'
s = 'https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv'

df = pd.read_csv(p)

for idx, row in df.iterrows():
    total = 0
    total +=organ_failure_history_points(c, row['Organ Failure History'])
    total +=age_points(c, row['Age'])
    total +=temp_points(c, row['Temperature'])
    total +=ph_points(c, row['pH'])
    total +=heartrate_points(c, row['Heart Rate'])
    total +=respiratory_points(c,row['Respiratory Rate'])
    total +=sodium_points(c, row['Sodium'])
    total +=potassium_points(c, row['Potassium'])
    total +=creatinine_points(c, row['Acute Renal Failure'], row['Creatinine'])
    total +=hematocrit_points(c, row['Hematocrit'])
    total +=white_blood_points(c, row['White Blood Count'])
    total +=fio2_points(c, row['FiO2'],row['PaO2'],row['A-a Gradient'])
    total +=(15-row['Glasgow Coma Scale'])
#     total +=row['Glasgow Coma Scale']
    df.loc[idx,'TOTAL'] = total

scores = pd.read_csv(s)
df['SCORE'] = scores

In [109]:
df[df['TOTAL'] != df['SCORE']]

,Patient,Organ Failure History,Age,Temperature,pH,Heart Rate,Respiratory Rate,Sodium,Potassium,Creatinine,Acute Renal Failure,Hematocrit,White Blood Count,Glasgow Coma Scale,FiO2,PaO2,A-a Gradient,TOTAL,SCORE


In [106]:
df[df['Patient'] == 'E6127']

,Patient,Organ Failure History,Age,Temperature,pH,Heart Rate,Respiratory Rate,Sodium,Potassium,Creatinine,Acute Renal Failure,Hematocrit,White Blood Count,Glasgow Coma Scale,FiO2,PaO2,A-a Gradient,TOTAL,SCORE


In [107]:
df['TOTAL'].to_csv('Scores_corrected.csv', index=False)

In [69]:
import pandas as pd
scores = pd.read_csv('https://hds5210-2020.s3.amazonaws.com/Scores.csv')

In [70]:
c = 'apache.json'

for idx, row in df.iterrows():
    total = 0
    total +=organ_failure_history_points(c, row['Organ Failure History'])
    total +=age_points(c, row['Age'])
    total +=temp_points(c, row['Temperature'])
    total +=ph_points(c, row['pH'])
    total +=heartrate_points(c, row['Heart Rate'])
    total +=respiratory_points(c,row['Respiratory Rate'])
    total +=sodium_points(c, row['Sodium'])
    total +=potassium_points(c, row['Potassium'])
    total +=creatinine_points(c, row['Acute Renal Failure'], row['Creatinine'])
    total +=hematocrit_points(c, row['Hematocrit'])
    total +=white_blood_points(c, row['White Blood Count'])
    total +=fio2_points(c, row['FiO2'],row['PaO2'],row['A-a Gradient'])
    total +=(15-row['Glasgow Coma Scale'])
    print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
          total,
          organ_failure_history_points(c, row['Organ Failure History']),
          age_points(c, row['Age']),
          temp_points(c, row['Temperature']),
          ph_points(c, row['pH']),
          heartrate_points(c, row['Heart Rate']),
          respiratory_points(c,row['Respiratory Rate']),
          sodium_points(c, row['Sodium']),
          potassium_points(c, row['Potassium']),
          creatinine_points(c, row['Acute Renal Failure'], row['Creatinine']),
          hematocrit_points(c, row['Hematocrit']),
          white_blood_points(c, row['White Blood Count']),
          (15-row['Glasgow Coma Scale']),
          fio2_points(c, row['FiO2'],row['PaO2'],row['A-a Gradient'])))
#     break

35	5	0	4	2	3	1	3	4	2	4	2	2	3
31	2	0	4	3	0	4	4	1	0	4	2	4	3
47	5	3	4	4	2	2	3	4	4	0	4	8	4
34	5	0	2	4	4	1	0	3	2	2	4	6	1
44	5	0	4	1	2	4	4	3	0	4	1	13	3
35	0	0	4	4	0	0	4	4	2	1	2	11	3
31	5	0	4	2	2	4	4	4	0	0	2	3	1
49	5	5	4	3	4	2	0	4	6	4	4	8	0
40	5	0	4	3	4	3	4	3	4	2	2	5	1
48	5	6	4	4	0	3	4	1	0	4	4	13	0
42	0	3	1	4	4	2	4	2	3	4	4	9	2
43	5	0	4	4	3	4	0	4	2	4	1	11	1
32	0	0	4	0	3	0	3	0	2	4	4	11	1
41	0	0	4	3	3	3	2	4	0	4	4	11	3
42	5	0	4	4	2	4	4	3	6	1	4	3	2
49	5	6	2	4	4	0	4	4	4	2	2	10	2
37	2	0	4	4	3	0	3	0	2	1	2	14	2
37	5	6	4	3	3	0	3	4	2	0	2	5	0
38	2	0	4	2	2	3	4	0	4	1	4	8	4
43	2	0	2	4	3	1	4	4	8	4	0	9	2
41	2	2	4	2	0	1	4	1	4	4	2	11	4
31	5	0	0	0	3	4	4	1	6	4	0	2	2
22	0	2	4	4	0	1	4	0	0	1	4	1	1
38	5	0	4	1	0	2	4	4	2	4	0	11	1
30	5	0	4	4	2	0	4	4	2	0	2	1	2
41	5	0	4	4	2	4	3	4	4	2	2	6	1
41	2	3	0	3	4	3	4	3	6	4	2	4	3
34	5	0	4	1	2	2	4	4	3	2	2	5	0
46	5	5	4	4	3	3	4	4	2	0	0	8	4
40	2	0	4	3	3	1	3	4	3	2	2	12	1
47	5	3	0	4	3	0	4	1	8	4	2	10	3
36	2	0	4	4	3	4	4	3	0	4	4	2	2
43	5	0	4	2	4	0	4	4	0	2	2	13	3
41	5	5	4	4	0	0	4	4	3	4	4	2	2
4